### A short notebook to inspect the results of the punctuation correction

In [1]:
import numpy as np
import pickle
import pandas as pd

In [4]:
import textwrap

def pprint(long_string, width = 1150):
    # Use the textwrap module to pretty print the string
    pretty_string = textwrap.fill(long_string, width=width)
    print(pretty_string)


In [3]:
with open("data/punctuation_correction/corrected_transcripts_all.pkl", "rb") as file:
    co_transcripts = pickle.load(file)


In [5]:
def get_data(path): 
        merged_df = pd.read_csv(path)
        strings = merged_df['String']
        str_lst = strings.values

        vocab = merged_df['Title'].values
        identifier = merged_df['identifier']
        identifier_vocab = pd.DataFrame({'ID': identifier, 'Vocab': vocab})
        identifier_vocab = identifier_vocab.set_index('Vocab')['ID'].to_dict()
        return merged_df, str_lst, vocab, identifier_vocab, identifier


merged_df, str_lst, vocab, identifier_vocab, identifier = get_data('data/merged_data_for_AI.csv')

In [6]:
merged_df["co_transcripts"] = co_transcripts

### Lets compare the number of sentences in the original transcripts and in the corrected versions of them. We use the sumy tokenizer to seperate the strings into sentences. 

In [9]:
from sumy.nlp.tokenizers import Tokenizer as SumyTokenizer
sumy_tokenizer = SumyTokenizer("de")


In [11]:
def count_sentences(text):
    global sumy_tokenizer
    try:  
        sentences =  sumy_tokenizer.to_sentences(text)
        sentence_count = len(sentences)
    except Exception:
        sentence_count = np.nan
    
    return sentence_count


In [12]:
merged_df["#sentences_original"] = merged_df["transcript"].apply(count_sentences)
merged_df["#senteces_corrected"] = merged_df["co_transcripts"].apply(count_sentences)

In [13]:
diff = merged_df["#senteces_corrected"]/ merged_df["#sentences_original"]
diff = [x for x in diff if not (np.isnan(x) or np.isinf(x))]
print(np.mean(diff))

14.841014518595879


### the corrected texts have nealry 15 times the amount of sentences! Lets check in an example, how the original version and the corrected one differ. 

In [14]:
#index that is used. Change, to see another example
example_index = 0

In [15]:
#helper function that prints the text in a readable output and also the amount of sentences. 
def analyze_text(text):
    print("# Sentences: ", count_sentences(text))
    pprint(text)

In [16]:
analyze_text(co_transcripts[example_index])

# Sentences:  75
Hey Leute, willkommen zurück auf dem Kanal. heute werden wir unsere Videoserie fortsetzen, in der wir versuchen, schöne, flatternde Benutzeroberflächen mit Schritt-für-Schritt- Erklärungen nachzubilden. Letzte Woche haben wir diese erste Seite des Designs erstellt. heute werden wir an dieser arbeiten. Wenn also ein Benutzer auf einen dieser Projektordner tippt, wird diese Seite geöffnet. Ich habe diese Seite bereits hier erstellt und auch diese Klasse hinzugefügt, bei der es sich um ein zustandsbehaftetes Widget handelt, das diese gefaltete Eigenschaft aufweist, die den Namen des Ordners darstellt. das wird vom Benutzer angetippt, und hier haben wir die Build-Methode mit nft scaffold. Jetzt gehen wir zurück zur ersten Seite und fügen diesem Container einen Gestendetektor hinzu. damit der Benutzer zur zweiten Seite navigieren kann, Ändern wir den Rückgabetyp davon -Methode zu einem beliebigen Widget, und beim Tippen rufen wir den Navigator des Kontexts auf, um zur Proje

In [17]:
analyze_text(merged_df["transcript"][example_index])

# Sentences:  43
 Hey Leute, willkommen zurück auf dem Kanal, heute werden wir unsere Videoserie fortsetzen, in der wir versuchen, schöne flatternde Benutzeroberflächen mit Schritt-für-Schritt- Erklärungen nachzubilden. Letzte Woche haben wir diese erste Seite des Designs erstellt, heute werden wir an dieser arbeiten  Wenn also ein Benutzer auf einen dieser Projektordner tippt, wird diese Seite geöffnet. Ich habe diese Seite bereits hier erstellt und auch diese Klasse hinzugefügt, bei der es sich um ein zustandsbehaftetes Widget handelt, das diese gefaltete Eigenschaft aufweist, die den Namen des Ordners darstellt das wird vom Benutzer angetippt und hier haben wir die Build-Methode mit nft scaffold. Jetzt gehen wir zurück zur ersten Seite und fügen diesem Container einen Gestendetektor hinzu, damit der Benutzer zur zweiten Seite navigieren kann. Ändern wir den Rückgabetyp davon -Methode zu einem beliebigen Widget und beim Tippen rufen wir den Navigator des Kontexts auf, um zur Projekts